In [ ]:
%run data_preprocessing.ipynb
%run emotion_NN.ipynb
%run webcam_display.ipynb

In [ ]:
# For Keras NN model, we pass in model as 1
X_train, Y_train, X_test, Y_test = fer_data_preprocessing(1)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
model_NN = Keras_NN(X_train, Y_train)

In [ ]:
trained_model = model_NN.train_model()

In [ ]:
saved_model = model_NN.load_model()

In [9]:
get_webcam_feed(saved_model)

(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 

(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 48, 48)
(1, 